# Ejemplo de BBDD: **Chroma**

Notebook generado a partir del documento proporcionado (secciones 6.x) y organizado en celdas de **Markdown** y **código** para su uso didáctico.

> Objetivo: practicar con Chroma (BD de vectores embebida) en local: instalación, colecciones, ingesta con metadatos, consultas por similitud con filtros, actualizaciones/borrados y persistencia. También se muestra uso con embeddings locales (sentence-transformers) y con APIs.


## 6.1. Introducción

En este apartado se trabaja, de forma práctica y paso a paso, con **Chroma**, una base de datos de vectores embebida que funciona directamente desde Python y persiste en disco sin desplegar servidores.

Partiremos de la instalación y el arranque local, crearemos colecciones con documentos y metadatos, y realizaremos búsquedas por similitud con filtros básicos. Veremos dos vías para generar embeddings (local con *sentence-transformers* y vía API) y comprobaremos la persistencia entre sesiones. A continuación, montaremos un mini-RAG con un PDF/Markdown: *chunking*, elección de `k` (`n_results`), y métricas sencillas como *hit rate*, **Recall@k** y **MRR** para valorar la calidad. Cerraremos comparando **coseno** frente a **L2** para entender cómo cambia el *ranking* según la métrica.


## 6.2. Instalación y arranque local

Chroma es una base de datos de vectores "embebida": se usa directamente desde el código (Python/JS) y puede persistir en disco sin necesidad de un servidor externo. Esto facilita practicar en local, crear colecciones, añadir documentos con metadatos y hacer consultas por similitud en minutos.

Aunque el modo embebido cubre la mayoría de prácticas, también es posible levantar Chroma como servicio HTTP.


### 6.2.1. Requisitos mínimos

- Python 3.9 o superior (recomendado 3.10/3.11).
- Espacio en disco para el índice (carpeta de persistencia).
- *(Opcional)* `sentence-transformers` u otro proveedor de embeddings si se desea generar vectores automáticamente.


### 6.2.2. Instalación paso a paso (entorno aislado)

**Crear y activar un entorno virtual**

- macOS/Linux:
```bash
python3 -m venv .venv
source .venv/bin/activate
```

- Windows (PowerShell):
```powershell
py -m venv .venv
.\.venv\Scripts\Activate.ps1
```

**Instalar Chroma (y opcionalmente la librería de embeddings local)**
```bash
pip install --upgrade pip
pip install chromadb
# Opcional para generar vectores locales sin API:
pip install sentence-transformers
```

**Comprobación rápida (desde Python):**


In [8]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Comprobación rápida de versiones en un entorno con chromadb instalado
try:
    import chromadb, sys
    print('Chroma', getattr(chromadb, '__version__', 'desconocida'))
    print(sys.version)
except Exception as e:
    print('Aviso: recuerda instalar chromadb y (opcional) sentence-transformers en tu entorno.')
    print('Error:', e)


Chroma 1.4.0
3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]


**Recursos oficiales:**
- Documentación de Chroma: https://docs.trychroma.com/docs/overview/getting-started
- Chroma en la nube (opcional para pruebas): consultar su página oficial.


### 6.2.3. Primer arranque: cliente en memoria vs. persistente

- **En memoria** (para pruebas rápidas): los datos se pierden al cerrar el proceso.
- **Persistente**: guarda el índice en una carpeta y lo reaprovecha en siguientes ejecuciones.

**Ejemplo mínimo (persistente) con colección, inserción y consulta:**


In [11]:
import chromadb
from chromadb.utils import embedding_functions

# Ruta local donde se guardará el índice
client = chromadb.PersistentClient(path="./chroma_data")

# Opción A: embeddings locales (sin API)
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Crear (o recuperar) la colección con métrica de coseno
col = client.get_or_create_collection(
    name="demo_docs",
    metadata={"hnsw:space": "cosine"},
    embedding_function=ef
)

# Insertar documentos con metadatos e IDs
docs = [
    "Guía rápida de instalación de Python",
    "Buenas prácticas de control de versiones con Git",
    "Introducción a la búsqueda semántica"
]
metas = [
    {"tema": "python", "nivel": "básico"},
    {"tema": "git", "nivel": "básico"},
    {"tema": "vectores", "nivel": "medio"}
]
ids = ["doc1", "doc2", "doc3"]
col.add(documents=docs, metadatas=metas, ids=ids)
col.add(documents='Guia de Redes', metadatas={'tema': 'infraestructura', 'año': 2025}, ids='net_01')
# Consulta por texto (Chroma genera el embedding y busca por similitud)
res = col.query(query_texts=["¿Cómo empezar con Git?"], n_results=2)
print(res["ids"], res["metadatas"])


[['doc2', 'net_01']] [[{'tema': 'git', 'nivel': 'básico'}, {'tema': 'infraestructura', 'año': 2025}]]


**Puntos clave del ejemplo**

- `PersistentClient(path=...)` crea/usa la carpeta `./chroma_data` para el índice.
- `embedding_function` permite que Chroma convierta automáticamente cada texto a vector (si no se pasa, se pueden añadir vectores ya calculados vía `embeddings=`).
- Los **metadatos** (*payload*) se guardan junto al documento y se usan luego para filtrar.


### 6.2.4. Filtros por metadatos y upserts rápidos


In [12]:
# Añadir filtros en la consulta
res = col.query(
    query_texts=["instalación"],
    n_results=3,
    where={"tema": "python"}  # filtro por payload
)
print(res["ids"], res["metadatas"])
res2 = col.query(query_texts='configuración de servidores', n_results=2, where={'tema': 'infraestructura'})
print(res2)
# Actualizar o reinsertar (mismo id → upsert)
col.update(
    ids=["doc1"],
    documents=["Instalación de Python en macOS y Windows (actualizada)"],
    metadatas=[{"tema": "python", "nivel": "básico"}]
)

# Eliminar
col.delete(ids=["doc3"])


[['doc1']] [[{'tema': 'python', 'nivel': 'básico'}]]
{'ids': [['net_01']], 'embeddings': None, 'documents': [['Guia de Redes']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'año': 2025, 'tema': 'infraestructura'}]], 'distances': [[0.7950179576873779]]}


### 6.2.5. Estructura de carpetas y persistencia

- La carpeta de persistencia (por ejemplo, `./chroma_data/`) contendrá los ficheros del índice y tablas auxiliares (formato interno basado en DuckDB/Parquet).
- Es seguro copiarla o respaldarla como parte del proyecto; al volver a crear el `PersistentClient` con la misma ruta, la colección queda disponible.


### 6.2.6. Uso con embeddings externos (APIs)

Si ya existen vectores (por ejemplo, generados con una API), se pueden insertar directamente **sin** `embedding_function`.


In [13]:
import chromadb
client = chromadb.PersistentClient(path="./chroma_data")
col_pre = client.get_or_create_collection(name="precomputed", metadata={"hnsw:space":"cosine"})

# embeddings: lista de listas (floats), longitud = nº documentos, dimensión fija
col_pre.add(
    ids=["x1", "x2"],
    documents=["Texto A", "Texto B"],  # opcional si solo quieres buscar por vector
    embeddings=[[0.1, 0.2, 0.05], [0.05, -0.1, 0.2]],
    metadatas=[{"tag":"A"}, {"tag":"B"}]
)


> Recomendación: normalizar los vectores si se va a usar **coseno** o **inner product** con vectores unitarios.


### 6.2.7. Comprobaciones y problemas habituales

- *“No encuentra sentence-transformers”*: instalar con `pip install sentence-transformers` o, si no se quieren dependencias, añadir `embeddings=` precomputados.
- *“Resultados inconsistentes”*: mezclar documentos insertados con y sin normalización de vectores; usar siempre la misma métrica (`{"hnsw:space": "cosine"}`, `"l2"`, `"ip"`) en la colección.
- *“Carpeta crece mucho”*: eliminar documentos que ya no hagan falta y realizar backups/limpieza periódica de la carpeta de persistencia.


## 6.3. Colecciones, documentos y filtros

Guía, paso a paso, para crear una colección con persistencia en disco, ingerir documentos con metadatos y consultar por similitud con y sin filtros. Dos formas habituales de generar embeddings: local con **sentence-transformers** y mediante API (OpenAI).


### 6.3.1. Preparación rápida

- Entorno: Python 3.10+ (recomendado), `chromadb` instalado y, si se desea generar embeddings locales, `sentence-transformers`.
- Carpeta de persistencia: `./chroma_data` (se puede cambiar).
- Métrica por colección: usaremos **cosine** (lo más común para embeddings normalizados).


In [14]:
# 0) Cliente persistente y función de embeddings local
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.PersistentClient(path="./chroma_data")
ef_local = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"  # 384D; rápido y suficiente para prácticas
)

# 1) Crear (o recuperar) colección con métrica coseno
col = client.get_or_create_collection(
    name="curso_demo",
    metadata={"hnsw:space": "cosine"},
    embedding_function=ef_local
)


In [15]:
# # Alternativa con API (ejemplo OpenAI)
# import os
# from chromadb.utils import embedding_functions

# ef_openai = embedding_functions.OpenAIEmbeddingFunction(
#     api_key=os.getenv("OPENAI_API_KEY"),
#     model_name="text-embedding-3-small"  # como ejemplo
# )
# col_api = client.get_or_create_collection(
#     name="curso_demo_api",
#     metadata={"hnsw:space": "cosine"},
#     embedding_function=ef_openai
# )


### 6.3.2. Dataset de ejemplo (documentos + metadatos)


In [16]:
docs = [
    "Instalación de Python en Windows y macOS",
    "Primeros pasos con Git y buenas prácticas de commit",
    "Arrays y operaciones vectorizadas con NumPy",
    "Visualización de datos con Matplotlib: líneas y barras",
    "Limpieza de datos: valores nulos y outliers",
    "Búsqueda semántica con embeddings y bases vectoriales",
    "Flujo básico de RAG: recuperar, filtrar y reenrutar pasajes",
    "Control de versiones avanzado: ramas, merge y resolución de conflictos",
    "Introducción a pandas: Series y DataFrames",
    "Hiperparámetros en modelos clásicos de ML",
    "Transformers y embeddings de texto para búsqueda",
    "Métricas de evaluación: recall@k, precisión y MRR"
]
metas = [
    {"tema":"python", "nivel":"basico", "idioma":"es", "anio":2024},
    {"tema":"git", "nivel":"basico", "idioma":"es", "anio":2024},
    {"tema":"numpy", "nivel":"medio", "idioma":"es", "anio":2024},
    {"tema":"viz", "nivel":"basico", "idioma":"es", "anio":2024},
    {"tema":"datos", "nivel":"medio", "idioma":"es", "anio":2024},
    {"tema":"vectores", "nivel":"medio", "idioma":"es", "anio":2024},
    {"tema":"rag", "nivel":"medio", "idioma":"es", "anio":2024},
    {"tema":"git", "nivel":"avanzado", "idioma":"es", "anio":2023},
    {"tema":"pandas", "nivel":"basico", "idioma":"es", "anio":2023},
    {"tema":"ml", "nivel":"medio", "idioma":"es", "anio":2022},
    {"tema":"vectores", "nivel":"medio", "idioma":"es", "anio":2023},
    {"tema":"eval", "nivel":"basico", "idioma":"es", "anio":2022}
]
ids = [f"d{i+1}" for i in range(len(docs))]

col.add(documents=docs, metadatas=metas, ids=ids)
print("Ingesta completada:", len(ids), "documentos")


Ingesta completada: 12 documentos


### 6.3.3. Consultas básicas (texto → vector → NN) y filtrado simple


In [ ]:
# Top-3 más cercanos a una intención
q1 = col.query(query_texts=["¿Cómo crear y fusionar ramas con Git?"], n_results=3)
print(q1["ids"], q1["metadatas"])

# Con filtro: solo documentos de 'git'
q2 = col.query(query_texts=["merge de ramas"], n_results=3, where={"tema":"git"})
print(q2["ids"], q2["metadatas"])

# Con filtro combinado (AND): 'vectores' en 2023
q3 = col.query(query_texts=["búsqueda semántica"], n_results=3, where= {"$and": [
            {"tema": "vectores"},
            {"anio": 2023},
        ]})

print(q3["ids"], q3["metadatas"])

q4 = col.query(query_texts='seguridad perimetral', n_results=5, where={'$or': [{'tema': 'seguridad'}, {'año': 2024}]})


[['d2', 'd8', 'd5']] [[{'idioma': 'es', 'nivel': 'basico', 'tema': 'git', 'anio': 2024}, {'nivel': 'avanzado', 'idioma': 'es', 'anio': 2023, 'tema': 'git'}, {'idioma': 'es', 'anio': 2024, 'tema': 'datos', 'nivel': 'medio'}]]
[['d8', 'd2']] [[{'anio': 2023, 'nivel': 'avanzado', 'idioma': 'es', 'tema': 'git'}, {'tema': 'git', 'idioma': 'es', 'anio': 2024, 'nivel': 'basico'}]]
[['d11']] [[{'anio': 2023, 'idioma': 'es', 'nivel': 'medio', 'tema': 'vectores'}]]


### 6.3.5. Persistencia: cerrar, reabrir y consultar

Al usar `PersistentClient`, el índice queda en disco y puede reutilizarse desde otro script o una nueva sesión.


In [18]:
# En otro script/sesión (simulado aquí):
import chromadb
from chromadb.utils import embedding_functions

client2 = chromadb.PersistentClient(path="./chroma_data")
ef_local2 = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)
col2 = client2.get_or_create_collection(
    name="curso_demo", metadata={"hnsw:space":"cosine"}, embedding_function=ef_local2
)

res = col2.query(query_texts=["instalación de Python"], n_results=2)
print(res["ids"], res["metadatas"])


[['d1', 'd4']] [[{'idioma': 'es', 'tema': 'python', 'nivel': 'basico', 'anio': 2024}, {'tema': 'viz', 'anio': 2024, 'idioma': 'es', 'nivel': 'basico'}]]
